In [ ]:
from browser import Browser
import time
import pandas as pd
import os
os.makedirs("Data/", exist_ok=True)
os.makedirs("Data/Listing", exist_ok=True)

In [ ]:
a = Browser(True)
k = 0
count_fail = 0
while True:
    if k == 0:
        a.driver.get("https://www.jpx.co.jp/english/listing/stocks/new/index.html")
    else:
        if k < 10:
            text = "0" + str(k)
        else:
            text = str(k)
        a.driver.get(f"https://www.jpx.co.jp/english/listing/stocks/new/00-archives-{text}.html")

    a.scroll_to_bottom()
    time.sleep(3)

    try:
        soup = a.get_soup()
        table_div = soup.find("div", {"class":"component-normal-table"})
        list_table = table_div.find_all("table")
        for table in list_table:
            temp = pd.read_html(str(table))[0]
            try: data = pd.concat([data, temp], ignore_index=True)
            except: data = temp.copy()
        
        if len(data) == 0:
            raise
        
        data.to_csv(f"Data/Listing/{k}.csv", index=False)
        k += 1
        count_fail = 0
        data = None
        a.reset_browser()
    except:
        count_fail += 1
        if count_fail == 5:
            break
        a.reset_browser()